<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
def tokenize(doc):
    
    tokens = re.sub(r'[^a-zA-Z ^0-9]', '', doc)
    tokens = tokens.lower().split()
    
    return tokens

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [4]:
import re
import spacy
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [5]:
nlp = spacy.load("en_core_web_lg")

In [6]:
def clean_soup(df_column, spec_chars_remove = []):
    """
    Input: dataframe column and list of specific characters to remove, 
    
    Output: List of cleaned observations
    """
    
    df_column = df_column.copy()
    
    soupy = [BeautifulSoup(df_column[ii], 'lxml').get_text() 
             for ii in range(df_column.shape[0])
            ]
    
    for char in spec_chars_remove:
        soupy = [soupy[ii].replace(char, ' ') 
                 for ii in range(len(soupy))
                ]
        
    to_clean = ['[^A-Za-z ]+', '   ', '  ']
    
    for char in to_clean:
        soupy = [re.sub(char, ' ', soupy[ii]) 
                 for ii in range(len(soupy))
                ]
        
    df_feature = pd.Series([nlp(soupy[ii].lower().strip()) 
                            for ii in range(len(soupy))
                           ])
        
    for row in range(df_feature.shape[0]):
        
        tokens = [token.lemma_ for token in df_feature[row]]
        
        tokens_redo = []
    
        for ii in range(len(tokens)):
        
            if (len(tokens[ii]) > 2) & ~(tokens[ii] == '-PRON-'):
                tokens_redo.append(tokens[ii])
        
        df_feature[row] = " ".join(tokens_redo)
         
    return df_feature

In [7]:
yelp['text_cleaned'] = clean_soup(yelp['text'])

In [36]:
stop_words = nlp.Defaults.stop_words.union(['review', 'pron', 'star', 
                                            'place','food', 'time', 'good',
                                            'come', 'order', 'experience'
                                            'service', 'know', 'restaurant',
                                            'want', 'wait', 'till', 'think',
                                            'tell', 'look', 'minute', 'year',
                                            'find', 'thing', 'people', 'great',
                                            'need', 'price', 'work', 'service',
                                            'like', 'try' ,'love'
                                            ])

In [10]:
#vectorize cleaned text
tfidf = TfidfVectorizer(stop_words = stop_words, max_features = 5000)

dtm = tfidf.fit_transform(yelp['text_cleaned'])

dtm = pd.DataFrame(dtm.todense(), columns = tfidf.get_feature_names())

dtm.head()

/home/j_m/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,aaa,ability,able,absolute,absolutely,abundant,aburi,acai,accent,accept,...,yup,zach,zen,zero,zest,zip,zombie,zone,zoo,zucchini
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [12]:
fake_review = ["""This was the greatest place to have lunch, 
                    the atmosphere was great, food was great, 
                    service was outstanding. Cheap prices, 
                    great lunch deals"""]

fake = tfidf.transform(fake_review)

In [13]:
nearest_vectors, nearest_loc = nn.kneighbors(fake.todense())

for neighbor in list(nearest_loc):
    print(f"""Stars:\n {yelp['stars'].iloc[neighbor]}\n
                Review:\n {yelp['text'].iloc[neighbor][0:300]}\n
                Review Cleaned:\n {yelp['text_cleaned'].iloc[neighbor][0:300]}""")

Stars:
 6625    3
6311    5
6204    5
469     4
1192    5
7594    5
555     3
2199    3
7486    4
2318    5
Name: stars, dtype: int64

                Review:
 6625    Food and service are ok. There are much better...
6311    天氣很熱吃不下東西，今天我點了一個韓國冷面湯、餐後點了甜點，冰沙系列不會太甜膩，覺得店家很用...
6204    旅行でラスベガスに来ましたがネイルがはげてるのが気になり、探したお店でした。\n質問にも丁寧...
469     O  o  thenk 6nnn  .b  cgv  xx TV cvg  9 nvehxc...
1192    This place is so cute! It has a great atmosphe...
7594    Great place to hang out SteveO is awesome and ...
555     Chingu Korean BBQ's Lunch Specials for $5.99! ...
2199    Came for lunch and didn't see much of a crowd ...
7486    Their view is amazing. Go there for LUNCH TIME...
2318    Great experience! The customer service is outs...
Name: text, dtype: object

                Review Cleaned:
 6625     food and service there much well place for lunch
6311                                                     
6204                                                     
469     thenk nnn cgv cvg 

The reviews found by nearest neighbors model are all rated with 3 and higher stars, 
and they mostly use similar language to which I used in the fake review

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [14]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from scipy.stats import randint, uniform

In [15]:
X_train, X_val, y_train, y_val = train_test_split(yelp['text_cleaned'], yelp['stars'],
                                                  test_size=0.10, random_state=42, stratify = yelp['stars'])

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((9000,), (1000,), (9000,), (1000,))

In [16]:
vect = TfidfVectorizer(stop_words=stop_words)

rfc = RandomForestClassifier()

svd = TruncatedSVD(algorithm='randomized')

In [17]:
# pipline using latent semantic indexing

lsi = Pipeline([('vect', vect), 
                ('svd', svd)])

pipeline = Pipeline([('lsi', lsi),
                 ('clf', rfc)])

# The pipeline puts together a bunch fit then transform,fit then predict.
parameters = {
    'lsi__vect__ngram_range': [(1, 1), (1, 2)],
    'lsi__vect__max_df': [.95],
    'lsi__vect__min_df': [.01],
    'lsi__vect__max_features': [5000],
    'lsi__svd__n_components': [30],
    'clf__min_samples_leaf': [5, 24],
    'clf__n_estimators': [250],
    'clf__max_depth': [20, 40],
    'clf__max_features': [0.88, 0.98],
    'clf__class_weight': [None] 
}

grid_search = GridSearchCV(
    pipeline, 
    parameters,  
    cv = 3, 
    verbose = 10,
    n_jobs = -1
)
grid_search.fit(X_train, y_train)

print(f'Best score: {grid_search.best_score_}\n')
print(f'Best hyperparameters: \n{grid_search.best_params_}\n')

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done  42 out of  48 | elapsed:  6.5min remaining:   55.9s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  7.4min finished
/home/j_m/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Best score: 0.5633333333333334

Best hyperparameters: 
{'clf__class_weight': None, 'clf__max_depth': 40, 'clf__max_features': 0.88, 'clf__min_samples_leaf': 5, 'clf__n_estimators': 250, 'lsi__svd__n_components': 30, 'lsi__vect__max_df': 0.95, 'lsi__vect__max_features': 5000, 'lsi__vect__min_df': 0.01, 'lsi__vect__ngram_range': (1, 1)}



In [18]:
best_pipeline = grid_search.best_estimator_
best_pipeline.fit(X_train, y_train)

print(f'Validation Accuracy:\n{best_pipeline.score(X_val, y_val)}')

Validation Accuracy:
0.559


In [19]:
grid_pred = best_pipeline.predict(X_val)
print(classification_report(y_val, grid_pred))

              precision    recall  f1-score   support

           1       0.55      0.66      0.60       150
           2       0.13      0.03      0.04        76
           3       0.28      0.10      0.15       110
           4       0.39      0.30      0.34       218
           5       0.64      0.85      0.73       446

   micro avg       0.56      0.56      0.56      1000
   macro avg       0.40      0.39      0.37      1000
weighted avg       0.49      0.56      0.51      1000



In [20]:
fake_review_pred = best_pipeline.predict(fake_review)

print(f"""
The model predicts my fake review would give
the location a rating of {fake_review_pred[0]} stars.
""")


The model predicts my fake review would give
the location a rating of 5 stars.



this model does fairly well with 1 or 5-star values, but has trouble predicting the others. 
It is highly uncertain of what a 2 and 3-star review is and that's due to a few factors, mainly,
there is less data available to train, relative to the other ratings. Also, the 
wording between 1, 2, and 3-star reviews may not differ too much, causing confusion.

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [21]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary
from spacy.tokenizer import Tokenizer
import pyLDAvis.gensim
import warnings #for LDA warnings

Learn the vocubalary of the yelp data:

In [22]:
def make_tokens(df_feature, addl_stop_words = ['-PRON-']):
    """
    Input: Column of a dataframe/ Pandas Series, 
    stop words you'd like to add to nlp's defaults
    
    Output: List consisting of tokens for each observation
    
    Assumes: nlp object initialized as nlp
    """
    
    tokens = []
    tokenizer = Tokenizer(nlp.vocab)

    STOP_WORDS = nlp.Defaults.stop_words.union(addl_stop_words)

    for doc in tokenizer.pipe(df_feature, batch_size=500):

        doc_tokens = []

        for token in doc: 
            if token.text not in STOP_WORDS:
                doc_tokens.append(token.text.lower())

        tokens.append(doc_tokens)

    return tokens

In [37]:
yelp['text_tokens'] = make_tokens(yelp['text_cleaned'], stop_words)
yelp['text_tokens'].head()

0    [beware, fake, fake, fake, small, business, lo...
1    [lunch, togo, quick, staff, friendly, complain...
2    [vegas, dozen, step, foot, circus, circus, rea...
3    [night, close, street, party, actually, group,...
4    [bad, lunch, senior, pay, eat, hot, salad, noo...
Name: text_tokens, dtype: object

In [38]:
id2word = Dictionary(yelp['text_tokens'])
id2word.filter_extremes(no_below=5, no_above=0.95)

Create a bag of words representation of the entire corpus

In [39]:
corpus = [id2word.doc2bow(text) for text in yelp['text_tokens']]

Your LDA model should be ready for estimation: 

In [40]:
warnings.filterwarnings("ignore",category=DeprecationWarning)

lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=20,
                   workers=6,
                   num_topics = 3 # tuned after initial results of pyLDAvis
                  )

In [43]:
words = [re.findall(r'"([^"]*)"', t[1]) for t in lda.print_topics()]

topics = [' '.join(t[0:3]) for t in words]

for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
experience little drink

------ Topic 1 ------
don staff nice

------ Topic 2 ------
nice definitely don



Create 1-2 visualizations of the results

In [42]:
warnings.filterwarnings("ignore", category = FutureWarning)

pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.008952  0.004142       1        1  36.270844
1     -0.000520 -0.009100       2        1  32.401184
0     -0.008432  0.004958       3        1  31.327969, topic_info=    Category         Freq        Term        Total  loglift  logprob
465  Default   819.000000        wasn   819.000000  30.0000  30.0000
213  Default  1468.000000  experience  1468.000000  29.0000  29.0000
154  Default  1891.000000         don  1891.000000  28.0000  28.0000
72   Default  1753.000000        nice  1753.000000  27.0000  27.0000
103  Default  1165.000000       table  1165.000000  26.0000  26.0000
..       ...          ...         ...          ...      ...      ...
795   Topic3   332.556610         use  1117.618530  -0.0515  -5.8429
154   Topic3   355.353485         don  1891.026245  -0.5111  -5.7766
81    Topic3   332.222473        room  1125.175537  -0.0592  -5.8440
114   Topic3   332.247864         fry  1167.557129  -0.0961  -5.8439
570   Topic3   329.270264       taste  1154.820068  -0.0942  -5.8529

[280 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
4346      1  0.586039  accessory
4346      2  0.180320  accessory
4346      3  0.225400  accessory
4010      1  0.646549       alex
4010      2  0.198938       alex
...     ...       ...        ...
1532      2  0.554025       yard
1532      3  0.201464       yard
5405      1  0.248236     yearly
5405      2  0.248236     yearly
5405      3  0.620591     yearly

[621 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

The LDA shows us that there are 3 distinct topics, which might
explain the uncertainty in our model's decision making. We have
5 categories, where there are only 3 coherent topics. I would guess 
that the topics predicted by LDA are '1-star reviews', 
'2 & 3-star reviews', and '4 & 5-star reviews'

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)